In [ ]:
#packages inlezen
from netCDF4 import Dataset, chartostring
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time
import hkvsobekpy as hkv
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from tqdm import tqdm
%matplotlib inline

In [ ]:
#paden opgeven naar modellen

DHydro_Model_his="../03_Model/Projectgebied1D2D_basismodel/FlowFM/output/FlowFM_his.nc" #Project
DHydro_Model_map = "../03_Model/Projectgebied1D2D_basismodel/FlowFM/output/FlowFM_map.nc" #Project
Sobek_Model="../00_Data/sobekdata/SOBEK.lit/1/" #Project

opslaan = True

In [ ]:
#his en map bestand uit D-Hydro uitlezen
his = Dataset(DHydro_Model_his)
network = Dataset(DHydro_Model_map)

In [ ]:
#his #zet deze regel aan om te kijken welke parameters er in het D-Hydro _his.nc bestand zitten

In [ ]:
#network #zet deze regel aan om te kijken welke parameters er in het D-Hydro _map.nc bestand zitten

## Data uitlezen uit Sobek en D-Hydro voor waterstanden op observatiepunten

In [ ]:
#data uitlezen uit Sobek
his_file = Sobek_Model+"/CALCPNT.HIS" #locatie van his bestand
calcpnt = hkv.read_his.ReadMetadata(his_file) #inlezen van his bestand in HKVSobekPy module

#commands om parameters, timestamps en locaties op te halen uit Sobek bestanden
#calcpnt.GetParameters()
#calcpnt.GetTimestamps()
#calcpnt.GetLocations()

parameter = [s for s in calcpnt.GetParameters() if "Waterlevel" in s]

sobek = calcpnt.DataFrame()
sobek_1D_WL = sobek[parameter[0]]

#measurement stations uitlezen
his_file = Sobek_Model+"/MEASSTAT.HIS"

calcpnt2 = hkv.read_his.ReadMetadata(his_file)

meas_loc = calcpnt2.GetLocations()

parameter1 = [s for s in calcpnt2.GetParameters() if "Water" in s]
parameter2 = [s for s in calcpnt2.GetParameters() if "Discharge" in s]

sobek_meas = calcpnt2.DataFrame()
sobek_meas_WL= sobek_meas[parameter1[0]]
sobek_meas_Q= sobek_meas[parameter2[0]]

#D-Hydro 1D
node_id = chartostring(network['mesh1d_node_id'][:])
node_id = [nodeid.rstrip() for nodeid in node_id]

In [ ]:

ref_time = ' '.join(his['time'].units.split()[2:4])
print('reference time D-Hydro:',ref_time)
tijd = his['time'][:]

ref = datetime.datetime.strptime(ref_time, "%Y-%m-%d %H:%M:%S")

index_his = [ref + datetime.timedelta(seconds=each) for each in tijd]

locaties = chartostring(his['station_id'][:])
locaties = [str(locatie).rstrip() for locatie in locaties]

#waterstanden measurement stations
data = his['waterlevel'][:]
D_meas_WL = pd.DataFrame(data= data, index=index_his, columns = locaties )

#bij de merge functies blijven alleen de tijdstappen behouden waarvoor in beide datasets een waarde is
df =D_meas_WL.merge(sobek_meas_WL, left_index=True, right_index=True, suffixes=['_dhydro','_sobek'])

#voegt de twee datasest samen, maar als de tijdstappen of start- of stoptijden niet gelijk zijn krijg je NaN in de dataframe
#dit wordt niet goed weergegeven bij het plotten
#df = D_meas_WL.join(sobek_meas_WL, lsuffix='_sobek', rsuffix='_dhydro')

In [ ]:
plot = 'plotly' #typ hier plotly om interactieve grafieken te genereren, anders worden er statische afbeeldingen genereert via matplotlib (makkelijker te exporteren)
#waterstanden measurement stations

for locatie in locaties:
    if plot == 'plotly':
        fig = px.line(df, x = df.index,y = [df.loc[:,locatie+'_sobek'],df.loc[:,locatie+'_dhydro']], title='Waterstand bij '+locatie,
                      labels={"value": "Waterstand [m NAP]",  "index": "Datum"},
                     template='plotly') #template aannpassen, opties zijn:     
        #['ggplot2', 'seaborn', 'simple_white', 'plotly',
        #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
        #'ygridoff', 'gridon', 'none']

        #verschillende opties voor aan en uitzetten van hoverinfo en opmaak ervan
        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(hovermode="x unified")
        
        if opslaan ==True:
            pio.write_image(fig,'../02_Validatie/SOBEK2DHYDRO/meetpunten/svg/meetpunt_'+ locatie +'.svg', engine='orca')
            fig.write_html('../02_Validatie/SOBEK2DHYDRO/meetpunten/html/meetpunt_'+ locatie +'.html', include_plotlyjs="cdn", full_html=False)
        
        fig.show()
    else:
        fig,ax = plt.subplots(figsize=(20, 10))
        ax.plot(df.index,df.loc[:,locatie+'_dhydro'])
        ax.plot(df.index,df.loc[:,locatie+'_sobek'])
        ax.grid()
        ax.tick_params(labelsize=16)
        ax.legend(['D-Hydro','Sobek 2'], fontsize = 16)
        ax.set_xlabel('Datum',fontsize=16)
        ax.set_ylabel('waterstand [m NAP]',fontsize=16)
        ax.set_title(locatie,fontsize=16)
        #fig.savefig(Gebied+'/meetpunt/waterstand bij meetpunt {}_V2.png'.format(locatie),bbox_inches='tight')

## Waterbalans

In [ ]:
#waterbalans termen uitlezen uit D-Hydro his bestand
in_bound = pd.DataFrame(data = his['water_balance_boundaries_in'][:], index=index_his,columns=['Boundaries in'])
out_bound = pd.DataFrame(data = his['water_balance_boundaries_out'][:], index=index_his, columns=['Boundaries out'])
total_bound = pd.DataFrame(data = his['water_balance_boundaries_total'][:], index=index_his)
storage = pd.DataFrame(data = his['water_balance_storage'][:], index=index_his, columns=['Storage'])
lateral = pd.DataFrame(data = his['water_balance_laterals_total'][:], index=index_his, columns=['Lateral disch. tot.'])
net_1D2D_in = pd.DataFrame(data = his['water_balance_exchange_with_1D_total'][:], index=index_his, columns=['Net 1D2D in'])
#sink = pd.DataFrame(data = his['water_balance_source_sink'][:], index=index_his)
totaal = pd.DataFrame(data = his['water_balance_total_volume'][:], index=index_his, columns=['Volume water system']) 
#time = pd.DataFrame(data = his['water_balance_total_volume'][:], index=index_his) 
in2D = pd.DataFrame(data = his['water_balance_exchange_with_1D_total'][:], index=index_his, columns=['in2D'])

#data = [his['water_balance_boundaries_in'][:], his['water_balance_boundaries_out'][:]]
data = [in_bound,out_bound,lateral,storage, totaal]
dhydro_balans = pd.concat(data, axis=1)

#waterbalans termen uitlezen Sobek his bestand
his_file = Sobek_Model+"/QWB.HIS"

calcpnt3 = hkv.read_his.ReadMetadata(his_file)

meas_loc = calcpnt3.GetLocations()

parameter1 = [s for s in calcpnt3.GetParameters() if "Volume" in s]
#parameter2 = [s for s in calcpnt2.GetParameters() if "Discharge" in s]

sobek = calcpnt3.DataFrame()
sobek_balans= sobek[parameter1[0]] *1000

sobek_balans



#bij de merge functies blijven alleen de tijdstappen behouden waarvoor in beide datasets een waarde is
df_balans =dhydro_balans.merge(sobek_balans, left_index=True, right_index=True, suffixes=['_dhydro','_sobek'])

#voegt de twee datasest samen, maar als de tijdstappen of start- of stoptijden niet gelijk zijn krijg je NaN in de dataframe
#dit wordt niet goed weergegeven bij het plotten
#df_balans =dhydro_balans.join(sobek_balans, lsuffix='_sobek', rsuffix='_dhydro')

In [ ]:
his

In [ ]:
plot = 'plotly' #typ hier plotly om interactieve grafieken te genereren, anders worden er statische afbeeldingen genereert via matplotlib (makkelijker te exporteren)
#waterstanden measurement stations

for i in ['Boundaries in', 'Boundaries out', 'Lateral disch. tot.','Storage', 'Volume water system']:
    if plot == 'plotly':
        fig = px.line(df_balans, x = df_balans.index,y = [df_balans.loc[:,i+'_sobek'],df_balans.loc[:,i+'_dhydro']], title=i,
                      labels={"value": "Volume [m3]",  "index": "Datum"},
                     template='plotly') #template aannpassen, opties zijn:     
        #['ggplot2', 'seaborn', 'simple_white', 'plotly',
        #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
        #'ygridoff', 'gridon', 'none']

        #verschillende opties voor aan en uitzetten van hoverinfo en opmaak ervan
        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(hovermode="x unified")
        
        #wegschrijven van figuren
        if opslaan == True:
            pio.write_image(fig,'../02_Validatie/SOBEK2DHYDRO/waterbalans/svg/balans_'+ i +'.svg', engine='orca')
            fig.write_html('../02_Validatie/SOBEK2DHYDRO/waterbalans/html/balans_'+ i +'.html', include_plotlyjs="cdn", full_html=False)

        fig.show()
        

        
    else:
        fig,ax = plt.subplots(figsize=(20, 10))
        ax.plot(df.index,df.loc[:,locatie+'_dhydro'])
        ax.plot(df.index,df.loc[:,locatie+'_sobek'])
        ax.grid()
        ax.tick_params(labelsize=16)
        ax.legend(['D-Hydro','Sobek 2'], fontsize = 16)
        ax.set_xlabel('Datum',fontsize=16)
        ax.set_ylabel('waterstand [m NAP]',fontsize=16)
        ax.set_title(locatie,fontsize=16)
        #fig.savefig(Gebied+'/meetpunt/waterstand bij meetpunt {}_V2.png'.format(locatie),bbox_inches='tight')

## Vergelijking bij stuwen

In [ ]:
#vergelijking maken bij kunstwerken en sturing

#inlezen D-Hydro data
stuwen = chartostring(his['weirgen_id'][:])
stuwen = [stuw.rstrip() for stuw in stuwen]

HWZ_dhydro= pd.DataFrame(data = his['weirgen_s1up'][:], index=index_his, columns=stuwen)
Q_dhydro = pd.DataFrame(data = his['weirgen_discharge'][:], index=index_his, columns=stuwen)
kruin_dhydro = pd.DataFrame(data = his['weirgen_crest_level'][:], index=index_his, columns=stuwen)

#inlezen Sobek data
#measurement stations uitlezen
his_file = Sobek_Model+"/STRUC.HIS"
calcpnt4 = hkv.read_his.ReadMetadata(his_file)
stuwen_sobek =calcpnt4.GetLocations()

parameter1 = [s for s in calcpnt4.GetParameters() if "up" in s]
parameter2 = [s for s in calcpnt4.GetParameters() if "Discharge" in s]
parameter3 = [s for s in calcpnt4.GetParameters() if "Crest" in s]

sobek = calcpnt4.DataFrame()

HWZ_sobek= sobek[parameter1[0]]
Q_sobek= sobek[parameter2[0]]
kruin_sobek= sobek[parameter3[0]]

#bij de merge functies blijven alleen de tijdstappen behouden waarvoor in beide datasets een waarde is
df_HWZ =HWZ_dhydro.merge(HWZ_sobek, left_index=True, right_index=True, suffixes=['_dhydro','_sobek'])
df_Q =Q_dhydro.merge(Q_sobek, left_index=True, right_index=True, suffixes=['_dhydro','_sobek'])
df_kruin =kruin_dhydro.merge(kruin_sobek, left_index=True, right_index=True, suffixes=['_dhydro','_sobek'])

#voegt de twee datasest samen, maar als de tijdstappen of start- of stoptijden niet gelijk zijn krijg je NaN in de dataframe
#dit wordt niet goed weergegeven bij het plotten
# df_HWZ =HWZ_dhydro.join(HWZ_sobek, lsuffix='_sobek', rsuffix='_dhydro')
# df_Q =Q_dhydro.join(Q_sobek, lsuffix='_sobek', rsuffix='_dhydro')
# df_kruin =kruin_dhydro.join(kruin_sobek, lsuffix='_sobek', rsuffix='_dhydro')

In [ ]:
#Het kan lang duren om voor alle stuwen een grafiek te maken, daarom is het hier mogelijk om een sleectie te maken
#stuwen = stuwen[0:50] #eerste 50 stuwen
#stuwen =['S242WMW','S256BA','S211B'] # lijst met IDs

In [ ]:
for locatie in stuwen:
    if locatie in stuwen_sobek:
        fig = make_subplots(rows=3, cols=1,
                           shared_xaxes=True, subplot_titles=("HWZ", "Kruinhoogte","Debiet"), 
                            vertical_spacing=0.1)

        fig.append_trace(go.Scatter(
            x=df_HWZ.loc[:,locatie+'_sobek'].index,
            y=df_HWZ.loc[:,locatie+'_sobek'].values,
            name = 'HWZ_Sobek',
            legendgroup = '1'
        ), row=1, col=1)

        fig.append_trace(go.Scatter(
            x=df_HWZ.loc[:,locatie+'_dhydro'].index,
            y=df_HWZ.loc[:,locatie+'_dhydro'].values,
            name = 'HWZ_D-Hydro',
            legendgroup = '1'
        ), row=1, col=1)

        fig.append_trace(go.Scatter(
            x=df_kruin.loc[:,locatie+'_sobek'].index,
            y=df_kruin.loc[:,locatie+'_sobek'].values,
            name = 'Kruinhoogte_Sobek',
            legendgroup = '2'
        ), row=2, col=1)
        fig.append_trace(go.Scatter(
            x=df_kruin.loc[:,locatie+'_dhydro'].index,
            y=df_kruin.loc[:,locatie+'_dhydro'].values,
            name = 'Kruinhoogte_D-Hydro',
            legendgroup = '2'
        ), row=2, col=1)

        fig.append_trace(go.Scatter(
            x=df_Q.loc[:,locatie+'_sobek'].index,
            y=df_Q.loc[:,locatie+'_sobek'].values,
            name = 'Debiet_Sobek',
            legendgroup = '3'
        ), row=3, col=1)
        fig.append_trace(go.Scatter(
            x=df_Q.loc[:,locatie+'_dhydro'].index,
            y=df_Q.loc[:,locatie+'_dhydro'].values,
            name = 'Debiet_D-Hydro',
            legendgroup = '3'
        ), row=3, col=1)

        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(
            height=800, 
            width=1000, 
            title_text='Stuw '+locatie, 
            xaxis3_title = 'Datum',
            yaxis1_title = 'Waterstand [m NAP]',
            yaxis2_title = 'Kruinhoogte [m NAP]',
            yaxis3_title = 'Debiet [m3/s]',
            legend_tracegroupgap = 180,
            hovermode="x unified",
        #     yaxis1_range=[50, 90],
            yaxis2_range=[df_kruin.loc[:,locatie+'_dhydro'].values.min()-1, df_kruin.loc[:,locatie+'_dhydro'].values.max()+1], #schaal aangepast omdat verschillen vaak klein zijn
        #     yaxis3_range=[50, 90]
        )
        fig.update_traces(xaxis='x1')
        
        #resultaten wegschrijven
        if opslaan==True:
            pio.write_image(fig,'../02_Validatie/SOBEK2DHYDRO/stuwen/svg/stuw_'+ locatie +'.svg', engine='orca')
            fig.write_html('../02_Validatie/SOBEK2DHYDRO/stuwen/html/stuw_'+ locatie +'.html', include_plotlyjs="cdn", full_html=False)
        fig.show()